In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import umap.umap_ as umap
import hdbscan
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- 1. Konfigurasi dan Pemuatan Data ---
# Muat model BERT yang telah dilatih sebelumnya.
# 'all-MiniLM-L6-v2' adalah model yang cepat dan memberikan performa yang baik.
print("Memuat model SentenceTransformer...")
model = SentenceTransformer('all-MiniLM-L6-v2')

# Muat dataset Anda. Pastikan nama file CSV sudah sesuai.
# Ganti 'Costumer complaint narrative.csv' jika nama file Anda berbeda.
file_path = r'C:\Users\rapha\Documents\VSCode\Kaggle\Final Project - Compfest Academy Data Sciene\Datasets\Costumer complaint narrative.csv'
print(f"Membaca data dari {file_path}...")
try:
    df = pd.read_csv(file_path)
    # Mengambil kolom yang berisi narasi keluhan
    # Pastikan nama kolomnya adalah 'Consumer complaint narrative'
    complaints = df['Consumer complaint narrative'].dropna().astype(str).tolist()
    print(f"Berhasil memuat {len(complaints)} narasi keluhan.")
except FileNotFoundError:
    print(f"Error: File tidak ditemukan di '{file_path}'. Mohon periksa kembali nama dan lokasi file.")
    exit()
except KeyError:
    print("Error: Kolom 'Consumer complaint narrative' tidak ditemukan dalam file CSV.")
    print("Pastikan nama kolom sudah benar.")
    exit()


# --- 2. Membuat Embeddings ---
# Mengubah setiap narasi keluhan menjadi vektor numerik (embedding).
# Proses ini mungkin memakan waktu tergantung pada jumlah data dan kekuatan komputer Anda.
print("\nMembuat embeddings untuk setiap keluhan... (Ini mungkin memakan waktu)")
# Untuk demonstrasi, kita hanya akan menggunakan sebagian kecil data (misal: 1000 keluhan pertama)
# Hapus baris di bawah ini jika Anda ingin memproses seluruh dataset.
complaints_subset = complaints[:1000]
embeddings = model.encode(complaints_subset, show_progress_bar=True)
print(f"Berhasil membuat embeddings dengan shape: {embeddings.shape}")


# --- 3. Clustering untuk Menemukan Tema Tersembunyi ---
def find_hidden_themes(embeddings, complaints_data):
    """
    Mengelompokkan embeddings untuk menemukan tema keluhan menggunakan UMAP dan HDBSCAN.

    Args:
        embeddings (np.array): Array numpy dari embeddings keluhan.
        complaints_data (list): List berisi teks keluhan asli.

    Returns:
        pd.DataFrame: DataFrame yang berisi keluhan, ID cluster, dan representasi 2D.
    """
    print("\n--- Memulai Proses Clustering Tema ---")
    # Langkah 3a: Reduksi Dimensi dengan UMAP
    # UMAP akan mengurangi dimensi embeddings agar lebih mudah di-cluster.
    print("1. Melakukan reduksi dimensi dengan UMAP...")
    umap_embeddings = umap.UMAP(n_neighbors=15,
                                n_components=5,
                                min_dist=0.0,
                                metric='cosine',
                                random_state=42).fit_transform(embeddings)

    # Langkah 3b: Clustering dengan HDBSCAN
    # HDBSCAN akan mengelompokkan titik data yang berdekatan di ruang UMAP.
    # Keluhan yang tidak masuk ke cluster manapun akan diberi label -1 (noise).
    print("2. Mengelompokkan data dengan HDBSCAN...")
    clusterer = hdbscan.HDBSCAN(min_cluster_size=15,
                                metric='euclidean',
                                cluster_selection_method='eom').fit(umap_embeddings)

    # Membuat DataFrame untuk analisis
    result_df = pd.DataFrame(umap_embeddings, columns=['x', 'y', 'z', 'a', 'b'])
    result_df['cluster'] = clusterer.labels_
    result_df['complaint'] = complaints_data

    # Menampilkan ringkasan hasil clustering
    print("\n3. Hasil Clustering:")
    cluster_counts = result_df['cluster'].value_counts().reset_index()
    cluster_counts.columns = ['Cluster ID', 'Jumlah Keluhan']
    print(cluster_counts)
    print("\nCluster -1 merepresentasikan 'noise' atau keluhan yang tidak masuk ke dalam kelompok manapun.")

    return result_df

# Jalankan fungsi clustering
clustered_df = find_hidden_themes(embeddings, complaints_subset)

# --- 4. Analisis Hasil Cluster ---
# Fungsi untuk melihat contoh keluhan dari setiap cluster
def view_cluster_samples(df, cluster_id, n_samples=5):
    """Menampilkan contoh keluhan dari cluster tertentu."""
    print(f"\n--- Contoh Keluhan dari Cluster {cluster_id} ---")
    samples = df[df['cluster'] == cluster_id]['complaint'].head(n_samples).tolist()
    for i, sample in enumerate(samples):
        print(f"{i+1}. {sample[:300]}...") # Tampilkan 300 karakter pertama
    print("-" * 20)

# Tampilkan contoh untuk beberapa cluster yang signifikan (bukan noise)
significant_clusters = clustered_df['cluster'].value_counts()
significant_clusters = significant_clusters[significant_clusters.index != -1].head(3).index

for cluster_id in significant_clusters:
    view_cluster_samples(clustered_df, cluster_id)


# --- 5. Pencarian Semantik ---
def semantic_search(query, embeddings_db, original_texts, top_k=5):
    """
    Mencari teks yang paling mirip secara semantik dengan query.

    Args:
        query (str): Teks kueri pencarian.
        embeddings_db (np.array): Database embeddings dari semua keluhan.
        original_texts (list): List berisi teks keluhan asli.
        top_k (int): Jumlah hasil teratas yang ingin ditampilkan.

    Returns:
        pd.DataFrame: DataFrame berisi hasil pencarian.
    """
    print(f"\n--- Memulai Pencarian Semantik untuk: '{query}' ---")
    # Ubah query menjadi embedding
    query_embedding = model.encode([query])

    # Hitung cosine similarity antara query dan semua keluhan
    similarities = cosine_similarity(query_embedding, embeddings_db)[0]

    # Dapatkan indeks dari hasil dengan similarity tertinggi
    top_k_indices = np.argsort(similarities)[-top_k:][::-1]

    # Siapkan hasil
    results = []
    for idx in top_k_indices:
        results.append({
            'similarity': similarities[idx],
            'complaint': original_texts[idx]
        })

    print(f"Menampilkan {top_k} hasil teratas:")
    return pd.DataFrame(results)

# Contoh penggunaan pencarian semantik
search_query = "incorrect information on my credit report about a medical bill"
search_results = semantic_search(search_query, embeddings, complaints_subset)

# Tampilkan hasil pencarian
for index, row in search_results.iterrows():
    print(f"\nSkor Kemiripan: {row['similarity']:.4f}")
    print(f"Keluhan: {row['complaint']}")


Memuat model SentenceTransformer...
Membaca data dari C:\Users\rapha\Documents\VSCode\Kaggle\Final Project - Compfest Academy Data Sciene\Datasets\Costumer complaint narrative.csv...
Berhasil memuat 59881 narasi keluhan.

Membuat embeddings untuk setiap keluhan... (Ini mungkin memakan waktu)


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Berhasil membuat embeddings dengan shape: (1000, 384)

--- Memulai Proses Clustering Tema ---
1. Melakukan reduksi dimensi dengan UMAP...


c:\Users\rapha\Documents\VSCode\Kaggle\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


2. Mengelompokkan data dengan HDBSCAN...

3. Hasil Clustering:
    Cluster ID  Jumlah Keluhan
0           -1             263
1            9             184
2            8              89
3            4              55
4           15              49
5           10              48
6           11              46
7            5              40
8           13              33
9            0              28
10          12              27
11           6              26
12           3              25
13           7              24
14          14              23
15           2              20
16           1              20

Cluster -1 merepresentasikan 'noise' atau keluhan yang tidak masuk ke dalam kelompok manapun.

--- Contoh Keluhan dari Cluster 9 ---
1. Barclays are harassing me by robocalling and hanging up upon answer. Violating tcpa laws as well. I did not consent to robocalls...
2. This company illegally repossessed my car with all of my possessions inside. 
They tried to get me to make 

c:\Users\rapha\Documents\VSCode\Kaggle\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\rapha\Documents\VSCode\Kaggle\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
